In [13]:
# %pip install coniferest

In [14]:
import sys
from pathlib import Path

import pandas as pd
from coniferest.label import Label
from coniferest.pineforest import PineForest
from coniferest.session import Session
from coniferest.session.callback import TerminateAfter, prompt_decision_callback
from lsst.daf.butler import Butler

sys.path.append("/sdf/data/rubin/user/kostya/linccf/internal/single_object")

In [15]:
repo = "/repo/main"
instrument = "LSSTComCam"
release = "v29_0_0_rc5"
collection = f"LSSTComCam/runs/DRP/DP1/{release}/DM-49865"

butler = Butler(repo, collections=collection)
hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats") / release

In [16]:
features_root = Path("object_lc_features")
data = pd.read_parquet(features_root, partitioning=None)
display(data)
ids = data.objectId.to_numpy()
X = data.query("r_psfMag > 22").drop(columns=["objectId"]).to_numpy()

,objectId,u_psfMag,g_psfMag,r_psfMag,i_psfMag,z_psfMag,y_psfMag,lg_anderson_darling_normal_flux_det_u_direct,duration_flux_det_u_direct,observation_count_flux_det_u_direct,...,bins_window1.0_offset0.0_excess_variance_flux_det_y_dia,bins_window1.0_offset0.0_arcsinh_kurtosis_flux_det_y_dia,bins_window1.0_offset0.0_linear_fit_slope_flux_det_y_dia,bins_window1.0_offset0.0_linear_fit_slope_sigma_flux_det_y_dia,bins_window1.0_offset0.0_linear_fit_reduced_chi2_flux_det_y_dia,bins_window1.0_offset0.0_mean_variance_flux_det_y_dia,bins_window1.0_offset0.0_arcsinh_skew_flux_det_y_dia,duration_src_y_dia,observation_count_src_y_dia,_healpix_29
0,4601772821591359576,30.000000,30.000000,30.000000,23.717869,30.000000,30.000000,-1.0,-1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,0.0,1258579486681748059
1,4601777219637870595,30.000000,30.000000,24.633097,23.601830,30.000000,30.000000,-1.0,-1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,0.0,1258579574796153301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837930,2226783725130103576,24.233543,23.644741,22.876316,22.242483,21.937317,21.577698,-1.0,-1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.196106,7.0,2528709617405752998
837931,2226783725130090405,24.772076,24.522953,24.273041,23.992460,23.355984,23.198605,-1.0,-1.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.196106,6.0,2528709618069617090


In [17]:
from single_object import make_figure
def decision_callback(metadata, data, session) -> Label:
    make_figure(metadata, butler, hats_path, image_type="direct")
    # return prompt_decision_callback(metadata, data, session)
    return Label.REGULAR

session = Session(
    data=X,
    metadata=ids,
    model=PineForest(random_seed=0),
    decision_callback=prompt_decision_callback,
    on_decision_callbacks=TerminateAfter(1),
)
session.run()

Is 1767038331138352465 anomaly? [y/N]:Is 4707774538601989969 anomaly? [y/N]:Is 2429977871989605108 anomaly? [y/N]:Is 1766994350673251280 anomaly? [y/N]:Is 2226827705595218862 anomaly? [y/N]:

In [18]:
session.known_labels

{np.int64(154537): <Label.REGULAR: 1>,
 np.int64(646438): <Label.REGULAR: 1>,
 np.int64(710695): <Label.REGULAR: 1>,
 np.int64(149422): <Label.REGULAR: 1>,
 np.int64(757788): <Label.REGULAR: 1>}